In [1]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
housing = fetch_california_housing()
import tensorflow as tf

In [2]:
X_train_full ,X_test,y_train_full,y_test = train_test_split(
    housing.data,housing.target
)
X_train,X_valid,y_train,y_valid = train_test_split(
    X_train_full,y_train_full
)

In [3]:
X_train.shape[1:]

(8,)

## 构建宽深神经网络

In [4]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30,activation="relu")(input_)
hidden2 = keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_],outputs=[output])

In [5]:
input_A = keras.layers.Input(shape=(5,), name="wide_input")
input_B = keras.layers.Input(shape=(6,), name="deep_input")
hidden1 = keras.layers.Dense(30,activation="relu")(input_B)
hidden2 = keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A,hidden2])
output = keras.layers.Dense(1,name="output")(concat)
aux_output = keras.layers.Dense(1,name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])

In [6]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")

X_train_A,X_train_B, = X_train[:,:5],X_train[:,2:]
X_valid_A,X_valid_B = X_valid[:,:5],X_valid[:,2:]
X_test_A,X_test_B = X_test[:,:5],X_test[:,2:]
X_new_A,X_new_B = X_test_A[:3],X_test_B[:3]

In [7]:
history = model.fit(
    [X_train_A,X_train_B],[y_train,y_train],epochs=20,
    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid])
)
total_loss,main_loss,aux_loss = model.evaluate(
    [X_test_A,X_test_B],[y_test,y_test]
)
y_pred_main,y_pred_aux = model.predict([X_new_A,X_new_B])

Epoch 1/20
363/363 [==============================] - 10s 16ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 2/20
363/363 [==============================] - 4s 11ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 4/20
363/363 [==============================] - 2s 6ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 5/20
363/363 [==============================] - 1s 4ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 6/20
363/363 [==============================] - 2s 4ms/step -

In [8]:
model.save("my_keras_model.h5")

In [9]:
model = keras.models.load_model("my_keras_model.h5")

In [10]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 30)           210         ['deep_input[0][0]']             
                                                                                                  
 wide_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 30)           930         ['dense_3[0][0]']                
                                                                                            

In [15]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5",save_best_only=True)

In [16]:
# history = model.fit(X_train,y_train,epochs=10)
history = model.fit(
    [X_train_A,X_train_B],[y_train,y_train],epochs=20,
    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),callbacks=[checkpoint_cb]
)
model = keras.models.load_model("my_keras_model.h5")

Epoch 1/60
363/363 [==============================] - 1s 2ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 2/60
363/363 [==============================] - 1s 3ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 3/60
363/363 [==============================] - 1s 2ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 4/60
363/363 [==============================] - 1s 3ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 5/60
363/363 [==============================] - 1s 3ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 6/60
363/363 [==============================] - 1s 2ms/step - lo

OSError: ignored

In [17]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(
    [X_train_A,X_train_B],[y_train,y_train],epochs=20,
    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),callbacks=[checkpoint_cb,early_stopping_cb]
)

Epoch 1/20
363/363 [==============================] - 3s 9ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_loss: nan - aux_output_loss: nan - val_loss: nan - val_output_loss: nan - val_aux_output_loss: nan
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - lo